In [2]:
import numpy as np
import pulp as pl
import pickle
import math

# offline lp rounding & fractional algorithms from 3.2
def load_pkl(ds):
    f = open(ds,'rb')
    data = pickle.load(f)
    return data

def offline_lp(ds):
    # load file(outside) and read data
    data=ds
    budgets=data[0]
    bids=data[1]
    queries=data[2]
    
    #get length of important variables
    n=len(budgets)
    r=len(bids)
    m=len(queries)
    
    
    # create (n+m)*(n*m) constraints matrix A
    #Firstly, for \sum_{i=1}^{n}x_ij <= 1, j \in [m]
    A = np.zeros((n+m, n*m))
    count=0
    for i in range(m):
        tmp=count
        while (count<m*n):
            A[i,count]=1
            count+=m
        count=tmp+1 
    
    
    #Then for \sum_{j=1}^{m}w_ij*x_ij <= 1, i \in [n]
    for i in range(n):
        for j in range(m):
            query = queries[j]
            A[m+i,i*m+j] = bids[i][query]
    
    # Next create (n+m)*1 constant vector B 
    B=[]
    i=0
    while(i<m+n):
        while(i<m):
            B.append(1)
            i+=1
        B.append(budgets[i-m])
        i+=1
    #And create (n+m)*2 bounds b
    b=[]
    for i in range(n*m):
        b.append([0, 1.0])

    #bids_queries=np.take(bids,queries,1).ravel()
    
    # Get a contiguous flattened 1-D array of combined bids and queries
    bids_queries=np.take(bids,queries,1).ravel()
    bq_length=len(bids_queries)
    
    
    #Get a list of variable:x1,x2,x3,...
    variables=[]
    xs=['x']*bq_length
    ns=list(range(1,bq_length+1))
    for i in range(bq_length):
        variables.append(pl.LpVariable(name=(xs[i]+str(ns[i])), lowBound=b[i][0], upBound = b[i][1]))
    

    model = pl.LpProblem("OptimizeModel", pl.LpMaximize)
    
    #constraints=[]
    #for i in range(len(A)):
    for i in (range(len(A))):
        e=pl.LpAffineExpression([(variables[j], A[i][j]) for j in range(bq_length) if A[i][j]!=0])
        bools=(e<=B[i])
        model+=bools
    #Add constraints to the LP model
        name=str(i)
        LPconstraints = pl.LpConstraint(e, pl.LpConstraintLE, name, B[i])
        model.addConstraint(LPconstraints)
    
    # solve the LP model
    objects = pl.lpDot(bids_queries, variables)
    model += objects
    model.solve(pl.getSolver('PULP_CBC_CMD'))
    
    values = [pl.value(variable) for variable in variables]
    obj_value = pl.value(objects)

    #print(obj_value, values)
    print(obj_value)
    
    return obj_value

def offline_lp_round(ds):
    # load file(outside) and read data
    data=ds
    budgets=data[0]
    bids=data[1]
    queries=data[2]
    
    #get length of important variables
    n=len(budgets)
    r=len(bids)
    m=len(queries)
    
    
    # create (n+m)*(n*m) constraints matrix A
    #Firstly, for \sum_{i=1}^{n}x_ij <= 1, j \in [m]
    A = np.zeros((n+m, n*m))

    count=0
    for i in range(m):
        tmp=count
        while (count<m*n):
            A[i,count]=1
            count+=m
        count=tmp+1 
    
    
    #Then for \sum_{j=1}^{m}w_ij*x_ij <= 1, i \in [n]
    for i in range(n):
        for j in range(m):
            query = queries[j]
            A[m+i,i*m+j] = bids[i][query]
    
    # Next create (n+m)*1 constant vector B 
    B=[]
    i=0
    while(i<m+n):
        while(i<m):
            B.append(1)
            i+=1
        B.append(budgets[i-m])
        i+=1
        
    #And create (n+m)*2 bounds b
    b=[]
    for i in range(n*m):
        b.append([0, 1.0])
   
    #bids_queries=np.take(bids,queries,1).ravel()
    
    # Get a contiguous flattened 1-D array of combined bids and queries
    bids_queries=np.take(bids,queries,1).ravel()
    bq_length=len(bids_queries)
    
    
    #Get a list of variable:x1,x2,x3,...
    variables=[]
    xs=['x']*bq_length
    ns=list(range(1,bq_length+1))
    for i in range(bq_length):
        variables.append(pl.LpVariable(name=(xs[i]+str(ns[i])), lowBound=b[i][0], upBound = b[i][1]))
    

    model = pl.LpProblem("OptimizeModel", pl.LpMaximize)
    #constraints=[]
    #for i in range(len(A)):
    for i in (range(len(A))):
        e=pl.LpAffineExpression([(variables[j], A[i][j]) for j in range(bq_length) if A[i][j]!=0])
        model+=e<=B[i]
        
    #Add constraints to the LP model
        name=str(i)
        LPconstraints = pl.LpConstraint(e, pl.LpConstraintLE, name, B[i])
        model.addConstraint(LPconstraints)
    
 
    # solve the LP model
    objects = pl.lpDot(bids_queries, variables)
    model += objects
    model.solve(pl.getSolver('PULP_CBC_CMD'))
    
    values = [pl.value(variable) for variable in variables]
    obj_value = pl.value(objects)

    #print(obj_value, values)
    print(obj_value)
    
    # print optimal revenues and chosen values
    optimal_revenues = pl.value(objects)
    chosen_values = []
    for i in range(len(variables)):
        chosen_values.append(pl.value(variables[i]))

    re=np.array(chosen_values).reshape(n,m)
    select=[]
    status=np.zeros(m)

    M=0
    #print(sum(budgets))


    for i in range(re.shape[1]):
        lst=[re[j][i] for j in range(re.shape[0])] 
        max_value_pos=np.argmax(lst)
        while (status[i]==0):
            if max(lst)==0:
                select.append(-1)
                status[i]=1

            if budgets[max_value_pos]>=bids[max_value_pos][queries[i]]:
                budgets[max_value_pos]-=bids[max_value_pos][queries[i]]
                select.append(max_value_pos)
                M += bids[max_value_pos][queries[i]]
                status[i]=1

            else:
                lst[max_value_pos]=0
                max_value_pos=np.argmax(lst)


    print(M)
    #print(budgets)
    
    return M, select    

In [3]:
#gen.py

def noisy_nz_1d(row,dec=2):
    """Adds small Gaussian noise, truncated to dec decimals,
    to each element of row.
    """
    noisy = []
    for x in row:
        if x == 0:
            noisy.append(x)
        else:
            noisy.append(round(x + random.gauss(0,0.1),dec))
    return noisy

def noisy_nz_2d(rows,dec=2):
    """Adds small Gaussian noise, truncated to dec decimals,
    to each element in each row of rows.
    """
    return [noisy_nz_1d(row,dec) for row in rows]

def ds0_gen(m):
    """Generates the 2 budgets, 2 bid vectors, and 2m queries
    for dataset ds0.

    Keyword arguments:
    n -- number of advertisers, an integer
    B -- budget for all advertisers, an integer
    """

    # one advertiser pays 1 for both keywords 
    #   and one pays 0.5 for only one
    bids = [ [1,1], [0.5,0] ]

    # suppose the queries alternate keywords (unrealistic)
    stream = [0,1]*m

    # first advertiser can afford all queries with first keyword,
    #   and second advertiser can afford all queries with second keyword.
    budgets = [m,m/2]

    return (budgets,bids,stream)


def ds1_gen(n,B):
    """Generates the n budgets, n bids vectors, and Bn queries
    for dataset ds1.

    Keyword arguments:
    n -- number of advertisers, an integer
    B -- budget for all advertisers, an integer
    """
    budgets = [B]*n

    # i-th adveriser bids 1 on first B*(i+1) keywords, 0 on rest
    bids = [ [1]*B*(i+1) + [0]*B*(n-i-1) for i in range(n)]

    # shuffle the advertisers due to deterministic construction
    random.shuffle(bids)

    # add small gaussian noise to bids
    bids = noisy_nz_2d(bids)

    # the queries are just each keyword once
    m = n*B
    stream = list(range(m))

    return (budgets,bids,stream)

def ds2_gen(n,B):
    """Generates the n budgets, n bid vectors, and Bn queries
    for dataset ds2.

    Keyword arguments:
    n -- number of advertisers, an integer
    B -- budget for all advertisers, an integer
    """

    # all budgets are B
    budgets = [B]*n

    mid = n//2

    prefix = [1]*B*mid
    ones = [1]*B
    zeros = [0]*B

    bids = []
    for i in range(n):
        if i < mid:
            # if i < n/2, then i-th adveriser
            #   bids 1 on keywords {Bi, ..., B(i+1)-1}
            #   and bids on no other keywords
            bid = zeros*i + ones + zeros*(n-i-1)
        
        else:
            # if i >= n/2, then i-th adveriser
            #   bids 1 on first Bn/2 keywords and {Bi, ..., B(i+1)-1}
            #   and bids on no other keywords
            i = i - mid
            bid = ones*mid + zeros*i + ones + zeros*(n-mid-i-1)

        bids.append(bid)

    # the queries are just each keyword once
    m = n*B
    stream = list(range(m))

    # shuffle the advertisers due to deterministic construction
    random.shuffle(bids)

    # randomly perturb the 1's
    bids = noisy_nz_2d(bids)

    return (budgets, bids, stream)

def ds3_gen(n,expo=1,f=1):
    """Generates the n bid vectors and f*n^2 queries
    for dataset ds3.

    Keyword arguments:
    n -- number of advertisers
    expo -- exponent to be used when generating the degrees
            of the queries (default 1.0)
    f -- a scale factor for the size of the stream w.r.t
            the n^2 keywords (default 1)
    """

    m = round(n**2)

    # initialize all bids to 0
    bids = [ [0]*m for _ in range(n) ]

    advertisers = list(range(n))

    # initialize degrees (# of bids) given by each advertiser to 0
    ad_degrees = [0]*n
    total_degree = 0

    for j in range(m):
        
        # get random degree d, 0 < d <= n
        while True:
            g = random.gauss(1,1)
            d = min(n, math.floor(math.exp(g)))
            break
            if d != 0:
                break

        # get weight distribution
        if total_degree == 0:
            # all degrees are 0, so weight all advertisers equally
            P = [1]*n
        else:
            # weight advertisers roughly proportional to their degree
            P = [(degree+1)**expo for degree in ad_degrees]

        # scale P so that it is a probability distribution
        sum_P = sum(P)
        P = [p/sum_P for p in P]

        # using P, sample d neighbors
        neighbors = np.random.choice(advertisers,d,replace=False,p=P)

        # get random value for j-th keyword to center bids around
        value = random.random()

        # generate the neighbors' bids
        for i in neighbors:
            bid = round(random.gauss(value,0.1),2)
            bids[i][j] = max(bid,0)
            ad_degrees[i] += 1
            total_degree += 1

    stream = list(range(m))

    bids = noisy_nz_2d(bids)

    # if scale factor f > 1, scale the query stream by f and
    #   randomly sample with replacement.
    if f > 1:
        stream = random.choices(stream, k=round(f*m))

    return (bids,stream)

def dump(name,obj):
    with open(f'{name}.pkl', 'wb') as f:
        pickle.dump(obj,f)

def load(name):
    with open(f'{name}.pkl', 'rb') as f:
        return pickle.load(f)

ds0 = ds0_gen(100)
ds1 = ds1_gen(20,20)
ds2 = ds2_gen(20,20)
ds3 = ds3_gen(20,1.3,10)

dump("ds0", ds0)
dump("ds1", ds1)
dump("ds2", ds2)
dump("ds3", ds3)

'''
Get budget vector for ds3:
1) pretend each advertiser has unlimited budget
2) set their budget equal to the amount spent during
    the online greedy algorithm with unlimited budget
'''

# pretend_budgets = [float('inf')]*20
# ds3_budgets = greedy(pretend_budgets,ds3[0],ds3[1])

'\nGet budget vector for ds3:\n1) pretend each advertiser has unlimited budget\n2) set their budget equal to the amount spent during\n    the online greedy algorithm with unlimited budget\n'

In [6]:
# offline greedy algorithm from 3.1

def load_pkl(ds):
    #load pickle file
    f = open(ds,'rb')
    data = pickle.load(f)
    return data


def offline_greedy(ds):
    # Input: dataset ds
    # Output: revenue and instance pair
    
    # Get parameters
    n = len(ds[0])             # the number of advertisers:
    r = len(ds[1][0])          # the number of keywords
    m = len(ds[2])             # the length of queries
    B = np.array(ds[0])        # bugets      
    W = np.array(ds[1])        # bid price matrix
    queries = np.array(ds[2])  # query
    
    # Initialization
    M = [0]*n
    select = [0]*m
    t_sold = []
    
    # Select from the largest bid
    w = np.sort(W,axis=None)[::-1]
    for bid in w:
        if bid > 0:
            # Multiplicity
            sel = np.array(np.where(W==bid))
        
            for i in range(sel.shape[1]):
                # Select by order of advertiser and keyword
                ad = sel[0,i]
                qu = sel[1,i]
            
                # Queries with that keyword, select by order of query
                t_fea = np.where(queries==qu)
                t_rem = np.setdiff1d(t_fea,t_sold)
                    
                for t in t_rem:
                    if B[ad]-M[ad] >= bid:
                        M[ad] += bid
                        select[t] = ad
                        t_sold.append(t)
    revenue = sum(M)
    return revenue, select
    

In [39]:
# experiment for ds0_type dataset,lp rounding
n=10
ds0 = ds0_gen(n)
dump("ds0", ds0)
offline_lp_round(ds0)

15.0
solved
15.0
15.0


(15.0, [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0])

In [40]:
# experiment for ds0_type dataset,greedy
ds0 = ds0_gen(n)
dump("ds0", ds0)
offline_greedy(ds0)

(10.0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [60]:
# experiment for ds1_type dataset,lp rounding
n=10
m=30
ds1 = ds1_gen(n,m)
dump("ds1", ds1)
offline_lp_round(ds1)

299.9999999980503
solved
300
295.8100000000002


(295.8100000000002,
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  6,
  1,
  1,
  1,
  1,
  1,
  1,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  9,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  0,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  -1,
  2,
  2,
  2,
  2,
  2,
  7,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  7,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  7,
  7,
  7,
  7,
  7,
  4,
  7,
  7,
  4,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  4,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  4,
  4,
  3,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  5,
  4,
  4,
  4,
  

In [61]:
# experiment for ds1_type dataset,greedy
ds1 = ds1_gen(n,m)
dump("ds1", ds1)
offline_greedy(ds1)

(228.45999999999995,
 [6,
  4,
  2,
  3,
  7,
  7,
  5,
  1,
  2,
  2,
  9,
  4,
  2,
  9,
  7,
  1,
  0,
  0,
  0,
  9,
  9,
  7,
  9,
  9,
  1,
  6,
  4,
  2,
  1,
  1,
  8,
  5,
  1,
  4,
  9,
  6,
  5,
  3,
  3,
  9,
  9,
  3,
  9,
  2,
  3,
  4,
  4,
  1,
  9,
  3,
  3,
  5,
  1,
  7,
  2,
  4,
  8,
  8,
  9,
  8,
  4,
  8,
  2,
  7,
  1,
  9,
  1,
  5,
  2,
  1,
  5,
  1,
  9,
  1,
  1,
  9,
  4,
  4,
  3,
  8,
  2,
  5,
  7,
  1,
  4,
  4,
  3,
  3,
  1,
  4,
  4,
  4,
  7,
  1,
  4,
  4,
  8,
  5,
  3,
  2,
  4,
  7,
  1,
  4,
  7,
  5,
  4,
  4,
  8,
  2,
  4,
  4,
  5,
  5,
  1,
  2,
  4,
  4,
  1,
  2,
  7,
  1,
  8,
  7,
  1,
  1,
  7,
  7,
  3,
  7,
  0,
  7,
  8,
  8,
  5,
  7,
  7,
  5,
  5,
  5,
  5,
  7,
  1,
  3,
  2,
  1,
  1,
  7,
  3,
  0,
  7,
  7,
  0,
  3,
  2,
  0,
  8,
  0,
  3,
  2,
  0,
  5,
  2,
  5,
  5,
  5,
  2,
  2,
  7,
  0,
  5,
  7,
  7,
  7,
  0,
  0,
  5,
  0,
  5,
  7,
  0,
  0,
  8,
  0,
  0,
  3,
  5,
  0,
  3,
  0,
  0,
  8,
  0,
  2,
  2,
  0,

In [68]:
# experiment for ds2_type dataset,lp rounding
n=10
m=30
ds2 = ds2_gen(n,m)
dump("ds2", ds2)
offline_lp_round(ds2)

298.72000000820026
solved
300
295.6900000000001


(295.6900000000001,
 [5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  -1,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  -1,
  0,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3

In [69]:
# experiment for ds2_type dataset,greedy
ds2 = ds2_gen(n,m)
dump("ds2", ds2)
offline_greedy(ds2)

(201.19,
 [7,
  7,
  9,
  6,
  9,
  2,
  1,
  2,
  9,
  1,
  2,
  1,
  6,
  9,
  9,
  4,
  4,
  4,
  2,
  7,
  2,
  2,
  2,
  1,
  4,
  2,
  6,
  6,
  2,
  7,
  7,
  0,
  6,
  7,
  0,
  1,
  0,
  0,
  4,
  0,
  6,
  1,
  6,
  0,
  0,
  1,
  0,
  4,
  4,
  7,
  4,
  4,
  0,
  1,
  6,
  1,
  0,
  9,
  9,
  0,
  5,
  7,
  5,
  4,
  1,
  9,
  5,
  4,
  4,
  9,
  1,
  6,
  1,
  9,
  7,
  1,
  6,
  4,
  4,
  9,
  1,
  7,
  9,
  6,
  9,
  5,
  4,
  5,
  5,
  5,
  1,
  3,
  3,
  6,
  9,
  3,
  3,
  1,
  3,
  4,
  9,
  7,
  3,
  7,
  4,
  7,
  3,
  7,
  9,
  3,
  3,
  6,
  3,
  4,
  1,
  6,
  7,
  1,
  1,
  1,
  7,
  1,
  9,
  8,
  8,
  8,
  8,
  4,
  8,
  8,
  8,
  4,
  4,
  8,
  6,
  7,
  1,
  9,
  7,
  7,
  8,
  8,
  8,
  9,
  6,
  8,
  8,
  7,
  9,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  7,
  7,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  9,
  9,
  9,
 

In [78]:
# experiment for ds3_type dataset,lp rounding
n=10
m=1.3
l=5
cds3=load_pkl('cds3.pkl')
ds3 = ds3_gen(n,m,l)
ds3=[cds3[0][:n],ds3[0],ds3[1]]
dump("ds3", ds3)
offline_lp_round(ds3)

259.80369565200107
solved
817.1000000000001
259.1799999999997


(259.1799999999997,
 [6,
  0,
  0,
  1,
  4,
  6,
  9,
  1,
  3,
  0,
  -1,
  0,
  6,
  6,
  3,
  3,
  3,
  6,
  4,
  4,
  6,
  6,
  9,
  7,
  -1,
  0,
  5,
  2,
  2,
  9,
  6,
  7,
  6,
  -1,
  0,
  2,
  9,
  -1,
  0,
  4,
  9,
  5,
  -1,
  0,
  6,
  3,
  7,
  6,
  6,
  3,
  9,
  -1,
  0,
  0,
  9,
  6,
  6,
  6,
  -1,
  0,
  5,
  9,
  1,
  6,
  -1,
  0,
  -1,
  0,
  7,
  6,
  7,
  8,
  4,
  2,
  9,
  9,
  -1,
  0,
  0,
  -1,
  0,
  9,
  5,
  9,
  6,
  6,
  9,
  6,
  3,
  -1,
  0,
  6,
  0,
  -1,
  0,
  7,
  6,
  9,
  2,
  0,
  -1,
  0,
  5,
  -1,
  0,
  5,
  6,
  1,
  3,
  6,
  6,
  -1,
  0,
  -1,
  0,
  6,
  0,
  3,
  6,
  6,
  0,
  3,
  7,
  0,
  0,
  6,
  1,
  -1,
  0,
  6,
  -1,
  0,
  8,
  1,
  1,
  6,
  0,
  -1,
  0,
  0,
  -1,
  0,
  1,
  0,
  2,
  -1,
  0,
  9,
  9,
  0,
  6,
  -1,
  0,
  6,
  4,
  -1,
  0,
  0,
  6,
  -1,
  0,
  1,
  -1,
  0,
  6,
  -1,
  0,
  0,
  3,
  7,
  3,
  6,
  3,
  2,
  0,
  0,
  6,
  6,
  7,
  6,
  6,
  -1,
  0,
  -1,
  0,
  3,
  -1,
  0,
  7,
  -1,

In [79]:
# experiment for ds3_type dataset,greedy
cds3=load_pkl('cds3.pkl')
ds3 = ds3_gen(n,m,l)
ds3=[cds3[0][:n],ds3[0],ds3[1]]
dump("ds3", ds3)
offline_greedy(ds3)

(200.52999999999992,
 [6,
  8,
  4,
  7,
  3,
  9,
  6,
  6,
  6,
  6,
  0,
  9,
  3,
  0,
  8,
  0,
  1,
  5,
  8,
  9,
  8,
  0,
  8,
  5,
  8,
  8,
  8,
  3,
  5,
  0,
  8,
  9,
  0,
  0,
  6,
  0,
  0,
  7,
  8,
  5,
  1,
  6,
  6,
  0,
  0,
  0,
  8,
  8,
  0,
  5,
  0,
  8,
  8,
  4,
  4,
  2,
  0,
  0,
  0,
  8,
  0,
  7,
  7,
  5,
  0,
  9,
  2,
  0,
  0,
  0,
  0,
  8,
  1,
  9,
  8,
  0,
  0,
  3,
  2,
  8,
  1,
  2,
  7,
  0,
  7,
  0,
  8,
  8,
  3,
  1,
  8,
  0,
  6,
  6,
  0,
  8,
  8,
  0,
  7,
  6,
  5,
  6,
  0,
  3,
  0,
  7,
  3,
  8,
  6,
  8,
  0,
  9,
  0,
  0,
  8,
  0,
  0,
  6,
  3,
  1,
  0,
  6,
  8,
  6,
  6,
  2,
  8,
  2,
  6,
  6,
  3,
  3,
  0,
  0,
  3,
  8,
  3,
  8,
  5,
  0,
  9,
  7,
  9,
  6,
  9,
  4,
  7,
  6,
  6,
  0,
  0,
  0,
  8,
  0,
  1,
  3,
  0,
  0,
  3,
  7,
  0,
  8,
  6,
  7,
  5,
  0,
  6,
  0,
  8,
  0,
  3,
  3,
  8,
  0,
  3,
  0,
  0,
  1,
  2,
  0,
  6,
  4,
  8,
  0,
  0,
  1,
  0,
  3,
  2,
  0,
  0,
  1,
  5,
  0,
  3,
  3,